In [1]:
import json
import os
import numpy as np

In [2]:
def generate_single_peaked_preference(num, peak_id = None):
    """
        input
            num: number of agent and items
            peak_id: numpy.array of size num
                must start from 0
        return
            preference of size num x num 
            e.g.,
             i1 i2 i3 i4 i5 i6 i7 i8
            [[8, 7, 6, 5, 4, 3, 2, 1], a1
             [7, 8, 7, 6, 5, 4, 3, 2], a2
             [6, 7, 8, 7, 6, 5, 4, 3], a3
             [3, 4, 5, 6, 7, 8, 7, 6], a4
             [2, 3, 4, 5, 6, 7, 8, 7], a5
             [1, 2, 3, 4, 5, 6, 7, 8], a6
             [1, 2, 3, 4, 5, 6, 7, 8], a7
             [1, 2, 3, 4, 5, 6, 7, 8]] a8
            and
            it<i{t+1}, at<a{t+1}
             
            

    """
    if peak_id is None:
        peak_id =np.random.choice(num,num,replace=True)
    # sort_id = np.argsort(peak_id)
    peak_id = np.expand_dims(peak_id,1)+1
    pref = np.repeat(np.expand_dims(np.linspace(1,num,num,dtype=int),0),num,axis=0)
    pref = num-np.abs(pref - peak_id)
    
    return pref

# def sort_preference_list(preference):
#     if preference.shape[0]==0:
#         return preference
#     pref_id = np.argmax(preference,axis=1)
#     return preference[pref_id,:], pref_id

def correct_idx(current_match,matches):
    store_match = current_match
    for match in matches[-1::-1]:
        if match[0] <= current_match[0]:
            current_match = current_match[0]+1,current_match[1]
        if match[1] <= current_match[1]:
            current_match = current_match[0],current_match[1]+1
    matches.append(store_match)
    return current_match, matches

def crawler_step(preference,matches):
    num_agents,num_items = preference.shape
    if num_agents == 0:
        return None
    # (1) if ot ≻it ot+1 holds for some t, let t∗ be the minimal such t
    # this means argmin_t(preference[t,t] > preference[t,t+1])
    
    # preference[t,t] > preference[t,t+1]
    screening = (preference.diagonal(offset=1)-preference.diagonal()[:-1])<0 # agent who (c1) does not want to move to larger one
    # check if (1) is possible
    if screening.sum() == 0:
        t_star = num_agents-1 # if no such agent (c1)
    else:
        t_star = np.argmax(screening) # if there is such agent (c1)
    agent_t_start_choice = np.argmax(preference[t_star]) # the agent's best choice
    next_preference = np.delete(preference, t_star, 0) # remove agent from the preference list
    next_preference = np.delete(next_preference, agent_t_start_choice, 1) # remove item from the preference list
    (t_star,agent_t_start_choice), matches = correct_idx((t_star,agent_t_start_choice),matches=matches) # correct index since the preference list is smaller
    return next_preference,t_star,agent_t_start_choice, matches

def crawler_algorithm(preference):
    matches = []
    result = crawler_step(preference,matches)
    preferences = [preference]
    preference = result[0]
    result_list = [result[1:3]]
    while result is not None:
        result = crawler_step(preference,matches)
        if result is not None:
            preference = result[0]
            preferences.append(result[0])
            matches = result[-1]
            result_list.append(result[1:3])
    return result_list, preferences

def verify_example_1():
    peak_id = np.array([7,2,7,6,7,3,5])-1
    num = len(peak_id)
    preference = generate_single_peaked_preference(num,peak_id)
    print(preference)
    # _, pref_idx = sort_preference_list(preference)
    result_list ,_= crawler_algorithm(preference)
    print(np.array(result_list)+1)


In [4]:
preference = generate_single_peaked_preference(20)
result_list,_ = crawler_algorithm(preference)
# verify one-to-one
print("One-to-one:",np.unique(np.array(result_list)[:,0]).shape[0] == len(result_list) and np.unique(np.array(result_list)[:,1]).shape[0] == len(result_list))


One-to-one: True


In [5]:
verify_example_1()

[[1 2 3 4 5 6 7]
 [6 7 6 5 4 3 2]
 [1 2 3 4 5 6 7]
 [2 3 4 5 6 7 6]
 [1 2 3 4 5 6 7]
 [5 6 7 6 5 4 3]
 [3 4 5 6 7 6 5]]
[[2 2]
 [6 3]
 [7 5]
 [4 6]
 [5 7]
 [3 4]
 [1 1]]


In [6]:

import plotly.graph_objects as go
import ipywidgets as widgets
import pandas as pd

In [9]:
peak_id = np.array([7,2,7,6,7,3,5])-1
num = len(peak_id)

preference = generate_single_peaked_preference(num,peak_id)
result_list, preferneces = crawler_algorithm(preference)
agent_left = [[i for i in range(1,num) if i not in np.array(result_list)[:,0]] for r in range(len(result_list))]

fig = go.FigureWidget(go.Figure(data=[go.Table(header=dict(values=list(range(1,8))),
                 cells=dict(values=preference,fill={'color':'#E8EDDF'}))
                     ]))
i = 0

int_range = widgets.IntSlider(value=0,max=num,min=0)
output2 = widgets.Output()

# display(int_range, output2)
def map_color(i):
    if i ==0:
        color_change_cells = np.array([])
    else:
        color_change_cells = np.array(result_list[:i])
    # print(color_change_cells)
    fill_color = []
    for row in range(num):
        row_colors = []
        for col in range(num):
            if len(color_change_cells) == 0:
                row_colors.append('#E8EDDF')
            else:
                if [col,row] in color_change_cells.tolist():
                    row_colors.append('#F5CB5C')
                elif col in color_change_cells[:,0] or row in color_change_cells[:,1]:
                    row_colors.append('#CFDBD5')
                else: # base
                    row_colors.append('#E8EDDF')
        fill_color.append(row_colors)
    # print(fill_color)
    return fill_color
def on_value_change(change):
    fill_color = map_color(change['new'])
    fig.update_traces( {'cells': {'fill': {'color': fill_color}},})
    with output2:
        print(result_list[:change['new']])

int_range.observe(on_value_change, names='value')
visual_tab  = widgets.VBox([
    int_range,fig])
display(visual_tab)

    'data': [{'cells': {'fill': {'color': '#E8EDDF'},…

In [8]:
widgets.Textarea(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False,
    layout=widgets.Layout(width='50%', height='80px')
)

Textarea(value='Hello World', description='String:', layout=Layout(height='80px', width='50%'), placeholder='T…

In [133]:
fig.__contains__

<bound method BaseFigure.__contains__ of FigureWidget({
    'data': [{'cells': {'fill': {'color': [['#ffffff'], ['#666666'], ['#ffffff'],
                                           ['#ffffff'], ['#ffffff'], ['#ffffff'],
                                           ['#ffffff'], ['#666666'], ['#660000'],
                                           ['#666666'], ['#666666'], ['#666666'],
                                           ['#666666'], ['#666666'], ['#ffffff'],
                                           ['#666666'], ['#ffffff'], ['#ffffff'],
                                           ['#ffffff'], ['#ffffff'], ['#ffffff'],
                                           ['#ffffff'], ['#666666'], ['#ffffff'],
                                           ['#ffffff'], ['#ffffff'], ['#ffffff'],
                                           ['#ffffff'], ['#ffffff'], ['#666666'],
                                           ['#ffffff'], ['#ffffff'], ['#ffffff'],
                                          